In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [12]:
company = pd.read_csv('公司文件.csv')
newsInfo1 = pd.read_csv('新闻基本信息表1.csv',on_bad_lines='skip', encoding='utf-8')
newsInfo2 = pd.read_csv('新闻基本信息表2.csv',on_bad_lines='skip', encoding='utf-8')
newsInfo3 = pd.read_csv('新闻基本信息表3.csv',on_bad_lines='skip', encoding='utf-8')
newsInfo=pd.concat([newsInfo1,newsInfo2,newsInfo3])
#截断正文数据，防止数据库存放时出现过长字段
newsInfo['NewsContent'] = newsInfo['NewsContent'].apply(lambda x: x[:255] if isinstance(x, str) else '')
news_security = pd.read_csv('新闻证券关联.csv')


C:\Users\王鸿轩\AppData\Local\Temp\ipykernel_17860\2867431352.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  newsInfo2 = pd.read_csv('新闻基本信息表2.csv',on_bad_lines='skip', encoding='utf-8')


In [2]:
db_user = 'user'
db_password = 'password'
db_host = 'localhost' 
db_port = '23306'
db_name = 'database'

# 创建数据库引擎
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')


In [4]:

# 将 DataFrame 写入数据库
table_name = 'company' 
company.to_sql(table_name, con=engine, if_exists='replace', index=False)
table_name = 'newsInfo'  
newsInfo.to_sql(table_name, con=engine, if_exists='replace', index=False)
table_name = 'news_security'  
news_security.to_sql(table_name, con=engine, if_exists='replace', index=False)


430956

In [5]:
# SQL 查询语句
query = """
SELECT 
    ni.NewsID,
    ni.DeclareDate,
    ni.Title,
    ni.FullDeclareDate,
    ni.Autor,
    ni.NewsContent,
    ni.NewsSource,
    si.Indcd,
    si.Indnme,
    si.Nindcd,
    si.Nindnme,
    si.Nnindcd,
    si.Nnindnme,
    si.IndcdZX,
    si.IndnmeZX
FROM 
    newsInfo ni
JOIN 
    news_security ns ON ni.NewsID = ns.NewsID
JOIN 
    company si ON ns.Symbol = si.Stkcd;
"""

df1 = pd.read_sql_query(query, engine)


In [6]:
df1.to_excel('SubmitData.xlsx', index=False)

In [7]:
#把数据存储为新表
table_name = 'news_industry'
df1.to_sql(table_name, con=engine, if_exists='replace', index=False)

424674

In [8]:
# sql查询 News Source 为 '东方财富网' 的数据
query = """
SELECT
    NewsID
FROM
    newsInfo
WHERE
    NewsSource = '东方财富网';
"""

dfcfw= pd.read_sql_query(query, engine)
dfcfw

,NewsID
0,105753522
1,105760527
2,105763045
3,105767900
4,105770846
...,...
3475,107509621
3476,107514902
3477,107514903
3478,107522480


In [17]:
index1 = pd.read_csv('指数权重1.csv')
index2 = pd.read_csv('指数权重2.csv')
index = pd.concat([index1,index2])
table_name = 'index_data'
index.to_sql(table_name, con=engine, if_exists='replace', index=False)

C:\Users\王鸿轩\AppData\Local\Temp\ipykernel_17860\2883424731.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  index2 = pd.read_csv('指数权重2.csv')


1205801

In [18]:
index.tail()

,Indexcd,Enddt,Stkcd,Constdnme,Weight
205796,H00160,2023-12-29,600018,上港集团,0.708
205797,H00160,2023-12-29,601618,中国中冶,1.130
205798,H00160,2023-12-29,600031,三一重工,4.229
205799,H00160,2023-12-29,601669,中国电建,2.177
205800,H00160,2023-12-29,600798,宁波海运,0.137


In [5]:
query = """
SELECT
    ns.NewsID,ni.Title
FROM
    index_data i
JOIN
    news_security ns ON i.Stkcd = ns.Symbol  
JOIN
    newsInfo ni ON ns.NewsID = ni.NewsID
WHERE
    i.Indexcd = 300;  
"""

hs300_news= pd.read_sql_query(query, engine)


In [6]:
#去掉重复数据
hs300_news.drop_duplicates(subset=['NewsID'], keep='first', inplace=True)

In [7]:
hs300_news

,NewsID,Title
0,105752709,又见高管变动！这次是长江副总裁罗国华 将出任上海证券总经理 拥有银保证丰富经验
51,105752820,这家券商帅将齐换！新董事长来自全国社保理事会 新总裁来自国信 在下什么棋？
103,105752822,中石油在上海首座油氢合建站正式投运
155,105752833,业绩暴跌、涉公务员培训贷、被监管立案 中公教育副总裁这样回应
207,105752835,中国电信携手中国信通院 深耕城市发展、产业升级和网信安全领域
...,...,...
34584931,107188963,下一站 芬兰！杉杉股份拟不超12.8亿欧元投资电池负极材料项目
34586037,107311525,杉杉股份：截至2023年9月30日 公司及公司控股子公司提供的对外担保总额约为212.29亿元
34587575,107438273,杉杉股份硅基负极逆势增长 中铝力推轻量化铝材 新能源汽车产业链企业链博会争雄
34588923,105938137,凯赛生物：2021年净利增32.82%至6.08亿元 拟10派4.5元


In [3]:
query = """
SELECT
    *
FROM
    news
"""

totalnews= pd.read_sql_query(query, engine)
totalnews.to_excel('totalnews.xlsx', index=False)
